In [99]:
import numpy as np
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().magic(u'matplotlib inline')
sns.set_style('whitegrid') 

import warnings
warnings.filterwarnings('ignore')

In [1]:
data = pd.read_stata('../論文csv/0722競標整理.dta',encoding='utf-8')
person = pd.read_stata('../論文csv/0722個人資訊.dta',encoding='utf-8')

NameError: name 'pd' is not defined

In [101]:
data.head()

,index,date,final_price,mer_id,mer_category,mer_name,new,suggested_price,winner,start_time,name,bids_in_auction,auto_in_auction,auction_profit,auction_profit96,auction_profit93
0,0,2016/03/24,380,18212,名牌配件,PORTER 和諧童趣 JUPITER,1,5850,沖沖沖,21:20,沖沖沖,140,126,1970,2010,2115
1,1,2016/03/24,380,18212,名牌配件,PORTER 和諧童趣 JUPITER,1,5850,沖沖沖,21:20,pat1011,166,63,-4150,-3984,-3859.5
2,2,2016/03/24,380,18212,名牌配件,PORTER 和諧童趣 JUPITER,1,5850,沖沖沖,21:20,阿寶一定發,2,0,-50,-48,-46.5
3,3,2016/03/24,380,18212,名牌配件,PORTER 和諧童趣 JUPITER,1,5850,沖沖沖,21:20,陳小妡,21,20,-525,-504,-488.25
4,4,2016/03/24,380,18212,名牌配件,PORTER 和諧童趣 JUPITER,1,5850,沖沖沖,21:20,qkfree168,1,0,-25,-24,-23.25


In [102]:
person.head()

,name,tt_reg_profit,tt_reg_profit96,tt_reg_profit93,tt_reg_auc,tt_reg_bids,tt_reg_win
0,沖沖沖,-115348,-103358,-94365.50,218,11990,77
1,pat1011,15627,17553,18997.50,48,1926,21
2,阿寶一定發,-125,-120,-116.25,3,5,0
3,陳小妡,-1675,-1608,-1557.75,14,67,0
4,qkfree168,-22941,-11237,-2459.00,2574,11704,181


In [103]:
def threegroup(x):
    if x <= 12:
        return 'fleet'
    if 12 < x < 42:
        return 'moderate'
    if x > 41:
        return 'persist'

In [104]:
person['tt_reg_auc'] = person['tt_reg_auc'].astype(int)
person['three_group'] = person['tt_reg_auc'].apply(threegroup)

In [105]:
# 分群人數
people = person.groupby('three_group')['name'].count()

In [106]:
# 下標次數
ttbids = person.groupby('three_group')['tt_reg_bids'].sum()

In [107]:
# 分群得標數
ttwins = person.groupby('three_group')['tt_reg_win'].sum()

In [108]:
# 分群利潤
ttprofit = person.groupby('three_group')['tt_reg_profit'].sum()
ttprofit96 = person.groupby('three_group')['tt_reg_profit96'].sum()
ttprofit93 = person.groupby('three_group')['tt_reg_profit93'].sum()

In [109]:
df = people.to_frame()
df = df.join(ttbids)
df = df.join(ttwins)
df = df.join(ttprofit)
df = df.join(ttprofit96)
df = df.join(ttprofit93)

In [110]:
df['pro_per_bid'] = df['tt_reg_profit'] / df['tt_reg_bids']

In [111]:
df

,name,tt_reg_bids,tt_reg_win,tt_reg_profit,tt_reg_profit96,tt_reg_profit93,pro_per_bid
three_group,,,,,,,
fleet,6636,373526,1201,-6050546,-5677020,-5396875.50,-16.198460
moderate,1007,498353,2528,-3929534,-3431181,-3057416.25,-7.885041
persist,382,1885735,11235,229573,2115308,3529609.25,0.121742


In [112]:
three_group = person[['name','three_group']]
three_group.to_csv('../論文csv/three_gruop.csv',encoding='utf-8')